In [2]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import numpy

Loading & prepping the dataset

In [3]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

# reshaping
X_train = X_train.reshape(len(X_train), 32, 32, 3)
Y_train = Y_train.reshape(len(Y_train), 1)
X_test = X_test.reshape(len(X_test), 32, 32, 3)
Y_test = Y_test.reshape(len(Y_test), 1)

# normalising the rgb values like 255 to something between 0 and 1
X_train = X_train / 255
X_test = X_test / 255

In [4]:
X_train[1,:]
print(X_train.shape)
print(X_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


The model

In [5]:
model = Sequential()

# 32 filters and 3x3
model.add(Conv2D(32, (3,3), activation= "relu", input_shape= (32, 32, 3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(32, (3,3), activation= "relu"))
model.add(MaxPooling2D(2,2))

# converts a multidimensional vector into a single dimensional vector
model.add(Flatten())

# hidden layer and output layer with 64 and 10 neurons respectively
model.add(Dense(64, activation="relu"))
model.add(Dense(10, activation="softmax"))

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model Compile

In [6]:
model.compile(loss= "sparse_categorical_crossentropy", optimizer= "adam", metrics= ["accuracy"])

Model Training/Fitting

In [7]:
model.fit(X_train, Y_train, epochs= 20, batch_size= 32)

Epoch 1/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.3841 - loss: 1.6998
Epoch 2/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.5845 - loss: 1.1784
Epoch 3/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6408 - loss: 1.0365
Epoch 4/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6684 - loss: 0.9519
Epoch 5/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6925 - loss: 0.8927
Epoch 6/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7097 - loss: 0.8375
Epoch 7/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7258 - loss: 0.7927
Epoch 8/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7438 - loss: 0.7372
Epoch 9/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7522 - loss: 0.7047
Epoch 10/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7651 - loss: 0.6742
Epoch 11/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7773 - loss: 0.6371
Epoch 12/20
1563/1563 ━━━━━━━━